## Hybrid Retriever- Combining Dense And Sparse Retriever\

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

/home/bjit/Desktop/Storage01/SelfDevelopment/Rag_Course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Environment setup
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key with error handling
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment variables")

print("✓ API keys loaded successfully")

✓ API keys loaded successfully


In [3]:
# Initialize Google Gemini embeddings model
# The gemini-embedding-001 model produces 3072-dimensional vectors
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [4]:
# Step 1: Sample documents
docs = [
    Document(page_content="LangChain helps build LLM applications."),
    Document(page_content="Pinecone is a vector database for semantic search."),
    Document(page_content="The Eiffel Tower is located in Paris."),
    Document(page_content="Langchain can be used to develop agentic ai application."),
    Document(page_content="Langchain has many types of retrievers.")
]

# Step 2: Dense Retriever (FAISS + Google Gemini)
embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
dense_vectorstore = FAISS.from_documents(docs, embedding_model)
dense_retriever = dense_vectorstore.as_retriever()

In [6]:
### Sparse Retriever(BM25)
sparse_retriever=BM25Retriever.from_documents(docs)
sparse_retriever.k=3 ##top- k documents to retriever

## step 4 : Combine with Ensemble Retriever
hybrid_retriever=EnsembleRetriever(
    retrievers=[dense_retriever,sparse_retriever],
    weight=[0.7,0.3]
)


In [7]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x797e75ff7bc0>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x797ed6f065d0>, k=3)], weights=[0.5, 0.5])

In [8]:
# Step 5: Query and get results
query = "How can I build an application using LLMs?"
results = hybrid_retriever.invoke(query)

# Step 6: Print results
for i, doc in enumerate(results):
    print(f"\n🔹 Document {i+1}:\n{doc.page_content}")


🔹 Document 1:
LangChain helps build LLM applications.

🔹 Document 2:
Langchain can be used to develop agentic ai application.

🔹 Document 3:
Langchain has many types of retrievers.

🔹 Document 4:
Pinecone is a vector database for semantic search.


### RAG Pipeline with hybrid retriever

In [9]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [11]:
# Step 5: Prompt Template
prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

## step 6-llm
# Initialize Google's Gemini model for response generation
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",    # Latest fast Gemini model
    temperature=0,               # Deterministic output for consistency
    max_tokens=None,            # Use model default
    timeout=None,               # No timeout limit
    max_retries=2,              # Retry failed requests
)

llm

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x797e75544980>, default_metadata=())

In [ ]:
### Create stuff Docuemnt Chain
document_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)

## create Full rAg chain
rag_chain=create_retrieval_chain(retriever=hybrid_retriever,combine_docs_chain=document_chain)
rag_chain

In [12]:
# Alternative: RAG Chain using LCEL (LangChain Expression Language) with "|" operator
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create RAG chain using LCEL syntax
rag_chain_lcel = (
    {"context": hybrid_retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✓ RAG chain created using LCEL syntax")
rag_chain_lcel

✓ RAG chain created using LCEL syntax


{
  context: EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x797e75ff7bc0>, search_kwargs={}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x797ed6f065d0>, k=3)], weights=[0.5, 0.5]),
  input: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the question based on the context below.\n\nContext:\n{context}\n\nQuestion: {input}\n')
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=0.0, max_retries=2, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x797e75544980>, default_metadata=())
| StrOutputParser()

In [ ]:
"""
Test the LCEL RAG Chain Implementation

This demonstrates how to use the LCEL-based RAG chain and handle its output.
Unlike traditional chains, LCEL with StrOutputParser returns a string directly.
"""

# Define the user query
query = "How can I build an app using LLMs?"

# Invoke the LCEL RAG chain - returns a string (not a dictionary)
response = rag_chain_lcel.invoke(query)

# Display the generated answer
# Note: response is already a string due to StrOutputParser()
print("✅ Answer:\n", response)

# Retrieve source documents separately for transparency
# LCEL chain doesn't return context in the response, so we call retriever directly
print("\n📄 Source Documents:")
source_docs = hybrid_retriever.invoke(query)

# Display each retrieved document with numbering
for i, doc in enumerate(source_docs):
    print(f"\nDoc {i+1}: {doc.page_content}")

✅ Answer:
 Based on the context, you can build an app using LLMs with the help of LangChain.

📄 Source Documents:

Doc 1: LangChain helps build LLM applications.

Doc 2: Langchain can be used to develop agentic ai application.

Doc 3: Langchain has many types of retrievers.

Doc 4: Pinecone is a vector database for semantic search.
